In [1]:
import time
import signal 
import requests
from time import sleep
import pandas
import numpy

In [2]:
def calculate_atr(ticker, window=14):
    payload = {'ticker': ticker}
    resp = s.get('http://localhost:9999/v1/securities/tas', params=payload)
    if not resp.ok:
        return None
    
    trades = resp.json()
    true_ranges = []

    for i in range(1, len(trades)):
        high = trades[i]['price']
        low = trades[i-1]['price']
        close = trades[i]['price']

        true_range = max(high - low, abs(high - close), abs(low - close))
        true_ranges.append(true_range)

    if len(true_ranges) < window:
        return None

    return sum(true_ranges[-window:]) / len(true_ranges[-window:])

def calculate_volatility(prices, window=10):
    if len(prices) < window:
        return None
    return sum([(p - sum(prices[-window:]) / window) ** 2 for p in prices[-window:]]) ** 0.5 / window

def adjust_risk_parameters(volatility):
    base_stop_loss = 0.02  # Minimum stop-loss percentage
    base_profit_target = 0.04  # Minimum profit-target percentage

    # Scale risk parameters with volatility
    stop_loss = base_stop_loss + (volatility * 0.5)  # Example scaling factor
    profit_target = base_profit_target + (volatility * 1.0)

    return stop_loss, profit_target


def decide_order_type(ticker, action, price, position):
    if ticker in ['CROW', 'DOVE']:  # Rebates for market orders
        order_type = 'MARKET'
    elif ticker in ['OWL', 'DUCK']:  # Rebates for limit orders
        order_type = 'LIMIT'
    else:
        order_type = 'LIMIT'  # Default to limit orders

    return {
        'ticker': ticker,
        'type': order_type,
        'quantity': ORDER_LIMIT,
        'price': price if order_type == 'LIMIT' else None,
        'action': action
    }

from math import gcd
from functools import reduce

def lcm(a, b):
    return abs(a * b) // gcd(a, b)

def calculate_lcm(prices):
    # Scale prices to integers (e.g., handle up to 2 decimal places)
    scaled_prices = [int(price * 100) for price in prices]  # Multiply by 100 to avoid floats
    lcm_scaled = reduce(lcm, scaled_prices)
    return lcm_scaled / 100  # Scale back to original form

    
def post_multiple_orders(ticker, action, total_quantity, price):
    remaining_quantity = total_quantity
    while remaining_quantity > 0:
        order_quantity = min(remaining_quantity, ORDER_LIMIT)
        s.post('http://localhost:9999/v1/orders', 
               params={'ticker': ticker, 'type': 'LIMIT', 
                       'quantity': order_quantity, 'price': price, 
                       'action': action})
        remaining_quantity -= order_quantity




In [5]:
s = requests.Session()
s.headers.update({'X-API-key': 'qqw'}) # Make sure you use YOUR API Key
ticker_list = ['OWL', 'CROW', 'DOVE', 'DUCK']

# global variables
MAX_LONG_EXPOSURE = 300000
MAX_SHORT_EXPOSURE = -100000
ORDER_LIMIT = 500
# Risk Management Parameters
STOP_LOSS_PERCENT = 0.05  # Exit position if price drops 5% below entry
PROFIT_TARGET_PERCENT = 0.10  # Exit position if price rises 10% above entry

# Store entry prices for each ticker
entry_prices = {}


def get_tick():
    resp = s.get('http://localhost:9999/v1/case')
    if resp.ok:
        case = resp.json()
        return case['tick'], case['status']
        
def price():
    prices = {}
    for ticker in ticker_list:
        resp = s.get(f'http://localhost:9999/v1/securities/tas?ticker={ticker}')
        if resp.ok:
            response = resp.json()
            if response:  # Ensure there's at least one trade
                prices[ticker] = response[0]['price']
                print(f'{ticker}:', prices[ticker])
            else:
                prices[ticker] = None  # No recent trades
        else:
            prices[ticker] = None  # Handle failed request
    return prices


def get_bid_ask(ticker):
    payload = {'ticker': ticker}
    resp = s.get ('http://localhost:9999/v1/securities/book', params = payload)
    if resp.ok:
        book = resp.json()
        bid_side_book = book['bids']
        ask_side_book = book['asks']
        
        bid_prices_book = [item["price"] for item in bid_side_book]
        ask_prices_book = [item['price'] for item in ask_side_book]
        
        best_bid_price = bid_prices_book[0]
        best_ask_price = ask_prices_book[0]
  
        return best_bid_price, best_ask_price

def get_time_sales(ticker):
    payload = {'ticker': ticker}
    resp = s.get ('http://localhost:9999/v1/securities/tas', params = payload)
    if resp.ok:
        book = resp.json()
        time_sales_book = [item["quantity"] for item in book]
        return time_sales_book

def get_position(ticker):
    resp = s.get('http://localhost:9999/v1/securities')
    if resp.ok:
        securities = resp.json()
        for security in securities:
            if security['ticker'] == ticker:
                return security['position']
    return 0  # Return 0 if ticker not found or request fails


def get_open_orders(ticker):
    payload = {'ticker': ticker}
    resp = s.get ('http://localhost:9999/v1/orders', params = payload)
    if resp.ok:
        orders = resp.json()
        buy_orders = [item for item in orders if item["action"] == "BUY"]
        sell_orders = [item for item in orders if item["action"] == "SELL"]
        return buy_orders, sell_orders

def get_order_status(order_id):
    resp = s.get ('http://localhost:9999/v1/orders' + '/' + str(order_id))
    if resp.ok:
        order = resp.json()
        return order['status']

def main():
    tick, status = get_tick()
    ticker_list = ['OWL', 'CROW', 'DOVE', 'DUCK']

    while status == 'ACTIVE':        
        # Fetch the latest prices for all tickers
        latest_prices = price()

        for ticker_symbol in ticker_list:
            position = get_position(ticker_symbol)
            best_bid_price, best_ask_price = get_bid_ask(ticker_symbol)
            current_price = latest_prices.get(ticker_symbol)

            if current_price is None:
                print(f"No recent trades for {ticker_symbol}. Skipping.")
                continue

            # Determine stop-loss and profit target thresholds dynamically (if enabled)
            recent_prices = get_time_sales(ticker_symbol)
            volatility = calculate_volatility(recent_prices)  # Optional: If volatility-based logic is used
            if volatility:
                stop_loss_percent, profit_target_percent = adjust_risk_parameters(volatility)
            else:
                stop_loss_percent, profit_target_percent = STOP_LOSS_PERCENT, PROFIT_TARGET_PERCENT

            print(f"Processing {ticker_symbol}: Position={position}, Current Price={current_price}")

            # Risk management: Check stop-loss and profit target
            if ticker_symbol in entry_prices and position != 0:
                entry_price = entry_prices[ticker_symbol]

                # Check stop-loss
                if position > 0 and current_price <= entry_price * (1 - stop_loss_percent):
                    print(f"Triggering stop-loss for {ticker_symbol} at {current_price}")
                    s.post('http://localhost:9999/v1/orders', 
                           params={'ticker': ticker_symbol, 'type': 'MARKET', 
                                   'quantity': abs(position), 'action': 'SELL'})
                    entry_prices.pop(ticker_symbol, None)

                # Check profit target
                elif position > 0 and current_price >= entry_price * (1 + profit_target_percent):
                    print(f"Triggering profit target for {ticker_symbol} at {current_price}")
                    s.post('http://localhost:9999/v1/orders', 
                           params={'ticker': ticker_symbol, 'type': 'MARKET', 
                                   'quantity': abs(position), 'action': 'SELL'})
                    entry_prices.pop(ticker_symbol, None)

                # For short positions
                elif position < 0 and current_price >= entry_price * (1 + stop_loss_percent):
                    print(f"Triggering stop-loss for short {ticker_symbol} at {current_price}")
                    s.post('http://localhost:9999/v1/orders', 
                           params={'ticker': ticker_symbol, 'type': 'MARKET', 
                                   'quantity': abs(position), 'action': 'BUY'})
                    entry_prices.pop(ticker_symbol, None)

                elif position < 0 and current_price <= entry_price * (1 - profit_target_percent):
                    print(f"Triggering profit target for short {ticker_symbol} at {current_price}")
                    s.post('http://localhost:9999/v1/orders', 
                           params={'ticker': ticker_symbol, 'type': 'MARKET', 
                                   'quantity': abs(position), 'action': 'BUY'})
                    entry_prices.pop(ticker_symbol, None)

            # Feebate-aware trading logic
            if position < MAX_LONG_EXPOSURE:
                order_params = decide_order_type(ticker_symbol, 'BUY', best_bid_price, position)
                print(f"Placing {order_params['type']} BUY order for {ticker_symbol}")
                s.post('http://localhost:9999/v1/orders', params=order_params)
                if ticker_symbol not in entry_prices:
                    entry_prices[ticker_symbol] = current_price

            if position > MAX_SHORT_EXPOSURE:
                order_params = decide_order_type(ticker_symbol, 'SELL', best_ask_price, position)
                print(f"Placing {order_params['type']} SELL order for {ticker_symbol}")
                s.post('http://localhost:9999/v1/orders', params=order_params)
                if ticker_symbol not in entry_prices:
                    entry_prices[ticker_symbol] = current_price

        # Cancel open orders and update market tick
        for ticker_symbol in ticker_list:
            print(f"Cancelling open orders for {ticker_symbol}")
            s.post('http://localhost:9999/v1/commands/cancel', 
                   params={'ticker': ticker_symbol})

        tick, status = get_tick()






if __name__ == '__main__':
    main()


OWL: 163.03
CROW: 90.24
DOVE: 23.78
DUCK: 67.52
Processing OWL: Position=0.0, Current Price=163.03
Placing LIMIT BUY order for OWL
Placing LIMIT SELL order for OWL
Processing CROW: Position=0.0, Current Price=90.24
Placing MARKET BUY order for CROW
Placing MARKET SELL order for CROW
Processing DOVE: Position=0.0, Current Price=23.78
Placing MARKET BUY order for DOVE
Placing MARKET SELL order for DOVE
Processing DUCK: Position=0.0, Current Price=67.52
Placing LIMIT BUY order for DUCK
Placing LIMIT SELL order for DUCK
Cancelling open orders for OWL
Cancelling open orders for CROW
Cancelling open orders for DOVE
Cancelling open orders for DUCK
OWL: 163.05
CROW: 90.41
DOVE: 23.88
DUCK: 67.61
Processing OWL: Position=500.0, Current Price=163.05
Placing LIMIT BUY order for OWL
Placing LIMIT SELL order for OWL
Processing CROW: Position=0.0, Current Price=90.41
Placing MARKET BUY order for CROW
Placing MARKET SELL order for CROW
Processing DOVE: Position=0.0, Current Price=23.88
Placing MARKET

KeyboardInterrupt: 